In [2]:
#######################################################################
# Copyright (C)                                                       #
# 2016 Shangtong Zhang(zhangshangtong.cpp@gmail.com)                  #
# 2016 Kenta Shimada(hyperkentakun@gmail.com)                         #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

from __future__ import print_function
import numpy as np

WORLD_SIZE = 5
A_POS = [0, 1]
A_PRIME_POS = [4, 1]
B_POS = [0, 3]
B_PRIME_POS = [2, 3]
discount = 0.9

world = np.zeros((WORLD_SIZE, WORLD_SIZE))

# left, up, right, down
actions = ['L', 'U', 'R', 'D']

actionProb = []
for i in range(0, WORLD_SIZE):
    actionProb.append([])
    for j in range(0, WORLD_SIZE):
        actionProb[i].append(dict({'L':0.25, 'U':0.25, 'R':0.25, 'D':0.25}))

nextState = []
actionReward = []
for i in range(0, WORLD_SIZE):
    nextState.append([])
    actionReward.append([])
    for j in range(0, WORLD_SIZE):
        next = dict()
        reward = dict()
        if i == 0:
            next['U'] = [i, j]
            reward['U'] = -1.0
        else:
            next['U'] = [i - 1, j]
            reward['U'] = 0.0

        if i == WORLD_SIZE - 1:
            next['D'] = [i, j]
            reward['D'] = -1.0
        else:
            next['D'] = [i + 1, j]
            reward['D'] = 0.0

        if j == 0:
            next['L'] = [i, j]
            reward['L'] = -1.0
        else:
            next['L'] = [i, j - 1]
            reward['L'] = 0.0

        if j == WORLD_SIZE - 1:
            next['R'] = [i, j]
            reward['R'] = -1.0
        else:
            next['R'] = [i, j + 1]
            reward['R'] = 0.0

        if [i, j] == A_POS:
            next['L'] = next['R'] = next['D'] = next['U'] = A_PRIME_POS
            reward['L'] = reward['R'] = reward['D'] = reward['U'] = 10.0

        if [i, j] == B_POS:
            next['L'] = next['R'] = next['D'] = next['U'] = B_PRIME_POS
            reward['L'] = reward['R'] = reward['D'] = reward['U'] = 5.0

        nextState[i].append(next)
        actionReward[i].append(reward)


In [3]:
import random

#generate a random trajectory for gridworld
def generate_episode(x, y, n_moves):
    #intialize traj with a state, action, and reward for each iteration in number of moves.
    traj = [{'state': [0,0], 'action': '', 'reward': 0.0} for i in range(n_moves)]
    #loop running as many times specified by n_moves
    for i in range(n_moves):
        #set the state of this trajectory iteration
        traj[i]['state'] = [y,x]
        #pick a random action for this trajectory iteration
        cur_act = random.choice(['L', 'U', 'R', 'D'])
        #set the action
        traj[i]['action'] = cur_act
        #calculate and set the reward from this state and action
        traj[i]['reward'] = actionReward[y][x][cur_act]
        #find the next state from the previous state and the action taken
        y,x = nextState[y][x][cur_act]
    return traj

In [4]:
def mcReward(trajectory):
    reward = 0
    #loop through trajectory starting at index j, where we found the first occurence
    for s in range(j, len(trajectory)):
        #sum all the rewards coming after s_j
        gamma = 0.9
        #with gamma
        #reward += (gamma**((s-j)))*trajectory[s]['reward']
        #without gamma
        reward += trajectory[s]['reward']
    return reward

In [5]:
#First-visit MC prediction

def firstVisitMCPrediction(num_episodes):
    value_state = [[0 for x in range(5)] for y in range(5)]
    #iterate through the number of episodes given
    for i in range(num_episodes):
        #get random starting point on gridworld map
        start_y = random.randint(0,4)
        start_x = random.randint(0,4)
        #generate episode trajectory with 100 random movies
        t = generate_episode(start_y, start_x, 100)
        
        #iterate through all possible states
        for y in range(5):
            for x in range(5):
                #current state = s_j
                s_j = [y,x]
                #iterate through the trajectory to find the first occurence of the current state (y, x)
                for j in range(len(t)):
                    #found the first occurence of s_j in t
                    if s_j == t[j]['state']:
                        #calculate the total reward proceeding this state.
                        reward = mcReward(t)

                        #i+1 is the number of episode that have occured before this episode including this episode
                    
                        #update the value state for this state, (start_y, start_x)
                        value_state[y][x] = value_state[y][x] + (1/(i+1)) * (reward - value_state[y][x])
    return value_state  

In [7]:
#run the first visit monte carlo prediction with 100 episodes
random
print(np.asarray(firstVisitMCPrediction(100)))

[[ 4.07669428  5.58571902  3.95773385  4.58189973 -1.9884335 ]
 [ 1.70558857  1.61139807  1.11991848  1.88265438 -2.05329679]
 [-1.8975151  -1.55383546 -0.75305341 -0.33490099 -1.79062979]
 [-1.69949553 -1.72362614 -2.48789646 -2.97364721 -3.670344  ]
 [-2.32045415 -1.5388675  -3.07750769 -4.32475979 -5.54331106]]
